In [1]:
import os
os.chdir("..")

from pprint import pprint
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb

from tuxai.dataset import Dataset, Columns
from tuxai.misc import config_logger
from tuxai.report import model_metrics, Report, FeatureImportanceReport

config_logger()
pd.options.display.max_rows = 300
pd.options.display.max_colwidth = 100


In [2]:
from tuxai.misc import cache
c = cache()
key = "fir_2022_12_21"
# del c[key]
if key in c:
    fir = c[key]
else:
    fir = FeatureImportanceReport()
    c[key] = fir


## Which options are importants, regardless of version? (always < top 30)

## 1 - no compression, with collinearity

In [3]:
fir.options_always_importants(target="vmlinux", rank=30, collinearity=True)

  0%|          | 0/7242 [00:00<?, ?it/s]

,options,4.13,4.15,4.20,5.00,5.04,5.07,5.08
27,nb_yes,0,1,1,0,0,0,0
21,"RANDOMIZE_BASE, X86_NEED_RELOCS",<NA>,0,0,1,1,1,1
20,RANDOMIZE_BASE,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,DEBUG_INFO,2,2,3,2,3,3,2
1,"AMD_MEM_ENCRYPT, DMA_COHERENT_POOL, ARCH_HAS_FORCE_DMA_UNENCRYPTED, DYNAMIC_PHYSICAL_MASK",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,3
15,"KASAN_OUTLINE, KASAN",4,3,2,<NA>,<NA>,<NA>,<NA>
14,"KASAN_GENERIC, KASAN_OUTLINE, KASAN",<NA>,<NA>,<NA>,3,2,4,<NA>
0,"AMD_MEM_ENCRYPT, ARCH_HAS_FORCE_DMA_UNENCRYPTED, DYNAMIC_PHYSICAL_MASK",<NA>,<NA>,<NA>,<NA>,5,2,<NA>
24,UBSAN_SANITIZE_ALL,3,4,4,4,4,8,8
7,DEBUG_INFO_REDUCED,5,5,5,5,6,5,5


## 2 - with compression, with collinearity

In [4]:

fir.options_always_importants(target="BZIP2-vmlinux", rank=30, collinearity=True)


  0%|          | 0/7257 [00:00<?, ?it/s]

,options,4.13,4.15,4.20,5.00,5.04,5.07,5.08
25,nb_yes,0,1,0,1,0,0,0
11,"KASAN_OUTLINE, KASAN",1,0,1,<NA>,<NA>,<NA>,<NA>
10,"KASAN_GENERIC, KASAN_OUTLINE, KASAN",<NA>,<NA>,<NA>,0,1,1,<NA>
22,UBSAN_SANITIZE_ALL,2,3,3,2,3,2,1
7,GCOV_PROFILE_ALL,3,2,2,3,2,5,3
12,KCOV_INSTRUMENT_ALL,4,5,5,4,6,3,2
8,IKHEADERS,<NA>,<NA>,<NA>,<NA>,5,4,5
16,"RANDOMIZE_BASE, X86_NEED_RELOCS",<NA>,4,4,5,4,7,6
19,UBSAN_ALIGNMENT,5,6,6,6,7,6,4
15,RANDOMIZE_BASE,6,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


## 3 - with compression, without collinearity : collinear options are lost

In [5]:
fir.options_always_importants(target="BZIP2-vmlinux", rank=30, collinearity=False)

  0%|          | 0/6811 [00:00<?, ?it/s]

,options,4.13,4.15,4.20,5.00,5.04,5.07,5.08
18,nb_yes,0,1,0,1,0,0,0
7,KASAN,1,0,1,0,1,1,<NA>
15,UBSAN_SANITIZE_ALL,2,3,3,2,3,3,1
5,GCOV_PROFILE_ALL,3,2,2,3,2,5,5
6,IKHEADERS,<NA>,<NA>,<NA>,<NA>,5,2,4
8,KCOV_INSTRUMENT_ALL,6,5,5,5,6,4,2
10,RANDOMIZE_BASE,5,4,4,4,4,6,7
12,UBSAN_ALIGNMENT,4,6,6,6,7,7,3
9,PRINTK,7,7,8,7,9,10,8
0,BINARY_PRINTF,11,10,7,8,8,9,6


## Which options are NEVER importants, regardless of version? (always > top 300)

## 1 - no compression

In [6]:
df = fir.options_never_importants(target="vmlinux", rank=300, collinearity=True)
df

  0%|          | 0/7242 [00:00<?, ?it/s]

,options,4.13,4.15,4.20,5.00,5.04,5.07,5.08
1887,I2C_ROBOTFUZZ_OSIF,<NA>,<NA>,<NA>,<NA>,<NA>,2191,<NA>
1902,I2C_XLR,<NA>,<NA>,<NA>,<NA>,<NA>,2188,<NA>
1927,IDLE_INJECT,<NA>,<NA>,<NA>,<NA>,<NA>,2181,<NA>
1932,IEEE802154_AT86RF230_DEBUGFS,<NA>,<NA>,<NA>,2166,<NA>,2179,<NA>
2379,LCD_OTM3225A,<NA>,<NA>,<NA>,2171,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...
4634,SPEAKUP_SYNTH_DUMMY,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,304
4709,STRICT_DEVMEM,<NA>,<NA>,<NA>,<NA>,<NA>,303,<NA>
868,CRYPTO_AEGIS128L,<NA>,<NA>,302,<NA>,<NA>,<NA>,<NA>
930,CRYPTO_LIB_CHACHA,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,301


In [7]:
df.dropna()

,options,4.13,4.15,4.20,5.00,5.04,5.07,5.08
365,BEFS_FS,1096,433,1526,1567,468,486,1569


## 2 - with compression

In [8]:
df = fir.options_never_importants(target="BZIP2-vmlinux", rank=300, collinearity=True)
df

  0%|          | 0/7257 [00:00<?, ?it/s]

,options,4.13,4.15,4.20,5.00,5.04,5.07,5.08
2044,I2C_MLXCPLD,<NA>,<NA>,<NA>,<NA>,<NA>,2124,<NA>
2048,I2C_MUX_GPIO,<NA>,<NA>,<NA>,<NA>,<NA>,2123,<NA>
2055,I2C_MV64XXX,<NA>,<NA>,<NA>,<NA>,<NA>,2122,<NA>
2062,I2C_PNX,<NA>,<NA>,<NA>,<NA>,<NA>,2118,<NA>
2063,I2C_QCOM_GENI,<NA>,<NA>,<NA>,<NA>,<NA>,2117,<NA>
...,...,...,...,...,...,...,...,...
3694,PM_SLEEP_DEBUG,<NA>,<NA>,<NA>,<NA>,304,<NA>,<NA>
3763,PWM_FSL_FTM,<NA>,<NA>,<NA>,<NA>,303,<NA>,<NA>
1621,FB_TFT_S6D1121,<NA>,<NA>,303,<NA>,<NA>,<NA>,<NA>
1593,FB_RIVA_BACKLIGHT,<NA>,301,<NA>,<NA>,<NA>,<NA>,<NA>


In [9]:
df.dropna()

,options,4.13,4.15,4.20,5.00,5.04,5.07,5.08
2715,MAX11100,1383,364,438,431,1618,1696,1988


# Options not always importants (stop or start beeing important)

In [11]:
fir.options_not_always_importants(target="vmlinux", best_rank=30, worst_rank=300, collinearity=True)

  0%|          | 0/7242 [00:00<?, ?it/s]

,options,4.13,4.15,4.20,5.00,5.04,5.07,5.08
3,"AMD_MEM_ENCRYPT, DMA_COHERENT_POOL, ARCH_HAS_FORCE_DMA_UNENCRYPTED, DYNAMIC_PHYSICAL_MASK",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,3
0,"AMD_MEM_ENCRYPT, ARCH_HAS_FORCE_DMA_UNENCRYPTED, DYNAMIC_PHYSICAL_MASK",<NA>,<NA>,<NA>,<NA>,5,2,<NA>
19,"SND_SOC_ALL_CODECS, SND_SOC_CS47L85, SND_SOC_CS47L92, SND_SOC_CS47L15, SND_SOC_CS47L90, SND_SOC_...",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,20
5,"BRANCH_PROFILE_NONE, FTRACE",<NA>,27,<NA>,<NA>,<NA>,<NA>,<NA>
10,EROFS_FS_ZIP,<NA>,<NA>,<NA>,<NA>,34,29,26
20,"SND_SOC_ALL_CODECS, SND_SOC_CS47L85, SND_SOC_CS47L92, SND_SOC_CS47L15, SND_SOC_CS47L90, SND_SOC_...",<NA>,<NA>,<NA>,<NA>,<NA>,31,<NA>
4,BRANCH_PROFILE_NONE,31,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
15,"SND_SOC_AD1980, SND_SOC_ALL_CODECS, SND_SOC_WM8782, SND_SOC_ADS117X, SND_SOC_L3, SND_SOC_ICS4343...",56,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
6,CONSTRUCTORS,90,76,61,30,118,45,<NA>
12,FTRACE,188,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
